## Prompt Optimization

Automatically create a prompt that is optimized to the model and evaluation metric.

In [1]:
#%pip install --quiet dspy

In [2]:
GEMINI="gemini-2.0-flash"
OPENAI="gpt-4o-mini"
CLAUDE="claude-3-7-sonnet-latest"

import os
from dotenv import load_dotenv
load_dotenv("../keys.env")
#assert os.environ["GEMINI_API_KEY"][:2] == "AI",\
#       "Please specify the GEMINI_API_KEY access token in keys.env file"
assert os.environ["ANTHROPIC_API_KEY"][:2] == "sk",\
       "Please specify the ANTHROPIC_API_KEY access token in keys.env file"
#assert os.environ["OPENAI_API_KEY"][:2] == "sk",\
#       "Please specify the OPENAI_API_KEY access token in keys.env file"

## Data

For simplicity, we'll hardcode the text of the marketing blurbs for an O'Reilly book.

In [3]:
mldp_text="""
The design patterns in this book capture best practices and solutions to recurring problems in machine learning. The authors, three Google engineers, catalog proven methods to help data scientists tackle common problems throughout the ML process. These design patterns codify the experience of hundreds of experts into straightforward, approachable advice.

In this book, you will find detailed explanations of 30 patterns for data and problem representation, operationalization, repeatability, reproducibility, flexibility, explainability, and fairness. Each pattern includes a description of the problem, a variety of potential solutions, and recommendations for choosing the best technique for your situation.

You'll learn how to:

Identify and mitigate common challenges when training, evaluating, and deploying ML models
Represent data for different ML model types, including embeddings, feature crosses, and more
Choose the right model type for specific problems
Build a robust training loop that uses checkpoints, distribution strategy, and hyperparameter tuning
Deploy scalable ML systems that you can retrain and update to reflect new data
Interpret model predictions for stakeholders and ensure models are treating users fairly
"""

In [4]:
# This is how much space is on the backcover.
TARGET_MAX_LINES = 15
TARGET_MAX_CHARS = 2400
TARGET_NUM_OBJECTIVES = 6

## Extract the pieces of the marketing description

In [5]:
import dspy
lm = dspy.LM(CLAUDE, api_key=os.environ['ANTHROPIC_API_KEY'])
#lm = dspy.LM(GEMINI, api_key=os.environ['GEMINI_API_KEY'])
dspy.configure(lm=lm)

In [6]:
from dataclasses import dataclass
from typing import List
from pydantic import Field

@dataclass
class Blurb:
    about_topic: str = Field(description="Why the topic of book is worth learning")
    about_book: str = Field(description="What book contains")
    target_audience: List[str] = Field(description="Roles such as Data Engineer, Data Analyst that the book is for")
    learning_objectives: List[str] = Field(description="4-6 learning objectives that complete the sentence: You will learn how to ___")
    
    def toMarketingCopy(self) -> str:
        nl = '\n'
        return f"""{self.about_topic}
        
{self.about_book}
        
{', '.join(list(self.target_audience))} will learn how to:
{nl.join(['* ' + objective for objective in self.learning_objectives])}
        """

In [7]:
class BlurbExtraction(dspy.Signature):
    text: str = dspy.InputField(desc="Text from backcover")
    blurb: Blurb = dspy.OutputField(desc="Information extracted from backcover text")

In [8]:
module = dspy.ChainOfThought(BlurbExtraction)
cover_info = module(text=mldp_text)
cover_info

Prediction(
    reasoning='Let me analyze the backcover text to extract the key information for the blurb:\n\n1. About the book:\n   - The book catalogs 30 design patterns for machine learning\n   - Written by three Google engineers\n   - Covers best practices and solutions to recurring problems in ML\n   - Includes detailed explanations of patterns for data representation, operationalization, repeatability, reproducibility, flexibility, explainability, and fairness\n\n2. About the topic:\n   - Machine learning design patterns are important because they codify expert experience into approachable advice\n   - These patterns help tackle common problems throughout the ML process\n   - They represent best practices from hundreds of experts\n\n3. Learning objectives (what you will learn how to):\n   - Identify and mitigate common challenges in training, evaluating, and deploying ML models\n   - Represent data for different ML model types\n   - Choose the right model type for specific proble

In [9]:
print(cover_info.blurb.toMarketingCopy())

Machine learning design patterns are essential because they codify the experience of hundreds of experts into straightforward, approachable advice. These patterns capture best practices and solutions to recurring problems throughout the ML process, helping practitioners avoid common pitfalls and implement robust ML systems.
        
This book presents 30 design patterns for machine learning, written by three Google engineers. It provides detailed explanations of patterns for data representation, operationalization, repeatability, reproducibility, flexibility, explainability, and fairness. Each pattern includes problem descriptions, potential solutions, and recommendations for choosing the best technique for your specific situation.
        
Data Scientists, Machine Learning Engineers, ML Practitioners, Data Professionals will learn how to:
* Identify and mitigate common challenges when training, evaluating, and deploying ML models
* Represent data effectively for different ML model typ

In [10]:
lm.inspect_history(n=1)





[2025-04-29T12:06:48.576705]

System message:

Your input fields are:
1. `text` (str): Text from backcover
Your output fields are:
1. `reasoning` (str)
2. `blurb` (Blurb): Information extracted from backcover text
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## text ## ]]
{text}

[[ ## reasoning ## ]]
{reasoning}

[[ ## blurb ## ]]
{blurb}        # note: the value you produce must adhere to the JSON schema: {"type": "object", "properties": {"about_book": {"type": "string", "description": "What book contains", "title": "About Book"}, "about_topic": {"type": "string", "description": "Why the topic of book is worth learning", "title": "About Topic"}, "learning_objectives": {"type": "array", "description": "4-6 learning objectives that complete the sentence: You will learn how to ___", "items": {"type": "string"}, "title": "Learning Objectives"}, "target_audience": {"type": "array", "description": "Roles such as Data Engineer, Dat

## Improve the marketing description components

In [11]:
## Improve the pieces
class BlurbImprovement(dspy.Signature):
    current_cover: Blurb = dspy.InputField(desc="Current information on book")
    about_topic: str = dspy.OutputField(desc="More catchy statement why topic is worth learning")
    about_book: str = dspy.OutputField(desc="More appealing (to target audience) description of book contents")
    target_audience: List[str] = dspy.OutputField(desc="more aspirational list of roles. Instead of programmer, say software engineer. Restrict to top 3.")
    learning_objectives: List[str] = dspy.OutputField(desc="Learning objectives rephrased or reordered to be more appealing to target audience. Exactly 6.")    
    
module = dspy.ChainOfThought(BlurbImprovement)
improved_cover = module(current_cover=cover_info.blurb)
improved_cover

Prediction(
    reasoning='The current cover information is solid but could be more compelling. I\'ll enhance it by:\n1. Making the "about topic" section more urgent and exciting by emphasizing how design patterns give practitioners a competitive edge\n2. Making the "about book" section more dynamic by highlighting the practical, transformative nature of the content\n3. Elevating the target audience descriptions to more aspirational titles\n4. Reframing the learning objectives to emphasize career advancement and practical outcomes rather than just skills acquisition',
    about_topic="Mastering machine learning design patterns is your competitive advantage in AI development. These battle-tested solutions distill the collective wisdom of industry leaders into actionable frameworks that dramatically accelerate your ML projects. By implementing these patterns, you'll avoid months of trial and error, sidestep costly pitfalls, and build production-ready ML systems that scale with enterprise

In [12]:
def make_blurb(d: dict):
    result = Blurb()
    result.about_topic = d['about_topic']
    result.about_book = d['about_book']
    result.target_audience = d['target_audience']
    result.learning_objectives = d['learning_objectives']
    return result

print(make_blurb(improved_cover.toDict()).toMarketingCopy())

Mastering machine learning design patterns is your competitive advantage in AI development. These battle-tested solutions distill the collective wisdom of industry leaders into actionable frameworks that dramatically accelerate your ML projects. By implementing these patterns, you'll avoid months of trial and error, sidestep costly pitfalls, and build production-ready ML systems that scale with enterprise demands.
        
Written by three Google engineering veterans, this essential guide unveils 30 powerful design patterns that form the backbone of successful machine learning systems at scale. You'll discover proven architectures for data representation, operationalization, repeatability, reproducibility, flexibility, explainability, and ethical AI implementation. Each pattern includes real-world case studies, practical code examples, and strategic decision frameworks to help you select the optimal approach for your specific challenges. This isn't just theory—it's your blueprint for b

In [13]:
lm.inspect_history(n=1)





[2025-04-29T12:06:48.604673]

System message:

Your input fields are:
1. `current_cover` (Blurb): Current information on book
Your output fields are:
1. `reasoning` (str)
2. `about_topic` (str): More catchy statement why topic is worth learning
3. `about_book` (str): More appealing (to target audience) description of book contents
4. `target_audience` (list[str]): more aspirational list of roles. Instead of programmer, say software engineer. Restrict to top 3.
5. `learning_objectives` (list[str]): Learning objectives rephrased or reordered to be more appealing to target audience. Exactly 6.
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## current_cover ## ]]
{current_cover}

[[ ## reasoning ## ]]
{reasoning}

[[ ## about_topic ## ]]
{about_topic}

[[ ## about_book ## ]]
{about_book}

[[ ## target_audience ## ]]
{target_audience}        # note: the value you produce must adhere to the JSON schema: {"type": "array", "items": {"ty

## Build an evaluator for the final output.

This is the metric that will be optimized

In [14]:
class BlurbScore(dspy.Signature):
    blurb: Blurb = dspy.InputField()
    topic_score: float = dspy.OutputField(desc="0-1 on how appealing the statement on why topic is worth learning. 0.9 means better than 90% of books.")
    contents_score: float = dspy.OutputField(desc="0-1 on how appealing (to target audience) the book contents are. 0.9 means better than 90% of books.")
    objectives_score: List[float] = dspy.OutputField(desc="0-1 score of how appealing each learning objective is to target audience. 0.9 means better than 90% of books.")

module = dspy.ChainOfThought(BlurbScore)
score_pred = module(blurb=make_blurb(improved_cover.toDict()))
score_pred

Prediction(
    reasoning='Let me evaluate this book blurb across the three dimensions:\n\nTopic Appeal:\nThe blurb positions machine learning design patterns as a "competitive advantage" and frames them as "battle-tested solutions" that represent "collective wisdom" from industry leaders. This is a compelling value proposition that speaks directly to practitioners\' desire for efficiency ("dramatically accelerate your ML projects") and risk reduction ("avoid months of trial and error, sidestep costly pitfalls"). The topic is timely and addresses a genuine pain point in the ML community - how to build systems that actually work in production. The framing of design patterns as a systematic approach to ML engineering borrows credibility from software engineering, where design patterns have proven valuable.\n\nContents Appeal:\nThe book promises 30 specific design patterns covering critical aspects of ML systems: data representation, operationalization, repeatability, reproducibility, fle

In [15]:
def calc_aggregate_score(blurb: Blurb, p: dspy.Prediction) -> float:   
    result = (p.topic_score * 10 + p.contents_score * 10 + sum(p.objectives_score)) / (20 + len(p.objectives_score))
    ## cut 0.1 for every line beyond MAX_LINES, etc.
    marketing_copy = blurb.toMarketingCopy()
    num_lines = len(marketing_copy.splitlines())
    num_chars = len(marketing_copy)
    num_objectives = len(blurb.learning_objectives)
    if num_lines > TARGET_MAX_LINES:
        result -= 0.1 * (num_lines - TARGET_MAX_LINES)
    if num_chars > TARGET_MAX_CHARS:
        result -= 0.01 * (num_chars - TARGET_MAX_CHARS)
    if num_objectives > TARGET_NUM_OBJECTIVES:
        result -= 0.1 * (num_objectives - TARGET_NUM_OBJECTIVES)
    if result < 0:
        result = 0
    return result

In [16]:
calc_aggregate_score(make_blurb(improved_cover.toDict()), score_pred)

0.8988461538461539

In [17]:
lm.inspect_history(n=1)





[2025-04-29T12:06:48.631518]

System message:

Your input fields are:
1. `blurb` (Blurb)
Your output fields are:
1. `reasoning` (str)
2. `topic_score` (float): 0-1 on how appealing the statement on why topic is worth learning. 0.9 means better than 90% of books.
3. `contents_score` (float): 0-1 on how appealing (to target audience) the book contents are. 0.9 means better than 90% of books.
4. `objectives_score` (list[float]): 0-1 score of how appealing each learning objective is to target audience. 0.9 means better than 90% of books.
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## blurb ## ]]
{blurb}

[[ ## reasoning ## ]]
{reasoning}

[[ ## topic_score ## ]]
{topic_score}        # note: the value you produce must be a single float value

[[ ## contents_score ## ]]
{contents_score}        # note: the value you produce must be a single float value

[[ ## objectives_score ## ]]
{objectives_score}        # note: the value you pro

## Combine into a module that can be optimized all at once

In [18]:
class BlurbPipeline(dspy.Module):
    def __init__(self):
        self.extract_info = dspy.ChainOfThought(BlurbExtraction)
        self.improve_blurb = dspy.ChainOfThought(BlurbImprovement)
        
    def forward(self, in_text: str) -> Blurb:
        cover_info = self.extract_info(text=in_text)
        improved_cover = self.improve_blurb(current_cover=cover_info.blurb)
        return make_blurb(improved_cover.toDict())
    
class ScorerPipeline(dspy.Module):
    def __init__(self):
        self.scorer = dspy.ChainOfThought(BlurbScore)
        
    def forward(self, blurb: Blurb) -> float:
        score_pred = self.scorer(blurb=blurb)
        return calc_aggregate_score(blurb, score_pred)

program = BlurbPipeline()
improved_blurb = program(in_text=mldp_text)
scorer = ScorerPipeline()
score = scorer(blurb=improved_blurb)
print(score, '\n', improved_blurb.toMarketingCopy())

0.8988461538461539 
 Mastering machine learning design patterns is your competitive advantage in AI development. These battle-tested solutions distill the collective wisdom of industry leaders into actionable frameworks that dramatically accelerate your ML projects. By implementing these patterns, you'll avoid months of trial and error, sidestep costly pitfalls, and build production-ready ML systems that scale with enterprise demands.
        
Written by three Google engineering veterans, this essential guide unveils 30 powerful design patterns that form the backbone of successful machine learning systems at scale. You'll discover proven architectures for data representation, operationalization, repeatability, reproducibility, flexibility, explainability, and ethical AI implementation. Each pattern includes real-world case studies, practical code examples, and strategic decision frameworks to help you select the optimal approach for your specific challenges. This isn't just theory—it's

## Do it several times and choose the best

This is about the best prompt for this one book

In [19]:
import dspy

def score_reward(args, blurb) -> float:
    scorer = ScorerPipeline()
    score = scorer(blurb=blurb)
    print(f"""**** {score}
{blurb.toMarketingCopy()}
******
    """)
    return score

optimized_pipeline = dspy.BestOfN(
    module=BlurbPipeline(),
    N=10,
    reward_fn=score_reward,
    threshold=0.95
)

In [20]:
print("****Orig****")
score_reward(None, cover_info.blurb) # prints

****Orig****
**** 0.9011538461538462
Machine learning design patterns are essential because they codify the experience of hundreds of experts into straightforward, approachable advice. These patterns capture best practices and solutions to recurring problems throughout the ML process, helping practitioners avoid common pitfalls and implement robust ML systems.
        
This book presents 30 design patterns for machine learning, written by three Google engineers. It provides detailed explanations of patterns for data representation, operationalization, repeatability, reproducibility, flexibility, explainability, and fairness. Each pattern includes problem descriptions, potential solutions, and recommendations for choosing the best technique for your specific situation.
        
Data Scientists, Machine Learning Engineers, ML Practitioners, Data Professionals will learn how to:
* Identify and mitigate common challenges when training, evaluating, and deploying ML models
* Represent data e

0.9011538461538462

In [21]:

best_blurb = optimized_pipeline(
    in_text=mldp_text
)

print("***Best****")
score_reward(None, best_blurb) # prints

**** 0.8988461538461539
Mastering machine learning design patterns is your competitive advantage in AI development. These battle-tested solutions distill the collective wisdom of industry leaders into actionable frameworks that dramatically accelerate your ML projects. By implementing these patterns, you'll avoid months of trial and error, sidestep costly pitfalls, and build production-ready ML systems that scale with enterprise demands.
        
Written by three Google engineering veterans, this essential guide unveils 30 powerful design patterns that form the backbone of successful machine learning systems at scale. You'll discover proven architectures for data representation, operationalization, repeatability, reproducibility, flexibility, explainability, and ethical AI implementation. Each pattern includes real-world case studies, practical code examples, and strategic decision frameworks to help you select the optimal approach for your specific challenges. This isn't just theory—i

0.9303846153846156

## Train on many more descriptions to create an optimized prompt

In [22]:
with open("blurbs.txt") as ifp:
    blurbs = ifp.read()
blurbs = blurbs.split("***")

# can create labeled also
# dspy.Example(context=context, question=question, answer=answer).with_inputs("context", "question"))
blurbs = [dspy.Example(in_text=b.strip()+"\n").with_inputs("in_text") for b in blurbs]
print(blurbs[2])

Example({'in_text': "Machine learning systems are both complex and unique. Complex because they consist of many different components and involve many different stakeholders. Unique because they're data dependent, with data varying wildly from one use case to the next. In this book, you'll learn a holistic approach to designing ML systems that are reliable, scalable, maintainable, and adaptive to changing environments and business requirements.\n\nAuthor Chip Huyen, co-founder of Claypot AI, considers each design decision--such as how to process and create training data, which features to use, how often to retrain models, and what to monitor--in the context of how it can help your system as a whole achieve its objectives. The iterative framework in this book uses actual case studies backed by ample references.\n\nThis book will help you tackle scenarios such as:\n\nEngineering data and choosing the right metrics to solve a business problem\nAutomating the process for continually develop

In [23]:
import dspy
from dspy.teleprompt import BootstrapFewShot

def evaluate_blurb(example, blurb, trace) -> float:
    scorer = ScorerPipeline()
    score = scorer(blurb=blurb)
    return score

optimizer = BootstrapFewShot(metric=evaluate_blurb)
optimized_pipeline = optimizer.compile(BlurbPipeline(), trainset=blurbs)
optimized_pipeline.save("optimized_pipeline", save_program=True)

 44%|████▍     | 4/9 [00:00<00:00, 16.08it/s]

Bootstrapped 4 full traces after 4 examples for up to 1 rounds, amounting to 4 attempts.


In [24]:
# Shows how to avoid cached result ...
lm_nocache =  dspy.LM(CLAUDE, api_key=os.environ['ANTHROPIC_API_KEY'], cache=False)
dspy.configure(lm=lm_nocache)
improved_blurb = optimized_pipeline(
    in_text=mldp_text
)
score_reward(None, improved_blurb) # prints
dspy.configure(lm=lm)

**** 0.9288461538461541
Machine learning design patterns represent the distilled wisdom of the industry's top practitioners—transforming years of hard-won experience into actionable blueprints for success. By mastering these patterns, you'll accelerate your ML development, avoid costly mistakes, and build solutions that stand up to real-world challenges. In a field where implementation details often determine success or failure, these battle-tested approaches give you a powerful competitive edge.
        
Authored by three Google engineering veterans, this essential guide reveals 30 proven design patterns that power some of the world's most sophisticated ML systems. Each pattern is meticulously documented with practical examples, implementation code, and strategic guidance on when to apply specific techniques. From data representation and operationalization to explainability and fairness, you'll gain insider knowledge that typically takes years of experience to acquire—all presented in

In [25]:
lm_nocache.inspect_history(n=3)





[2025-04-29T12:07:09.941046]

System message:

Your input fields are:
1. `text` (str): Text from backcover
Your output fields are:
1. `reasoning` (str)
2. `blurb` (Blurb): Information extracted from backcover text
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## text ## ]]
{text}

[[ ## reasoning ## ]]
{reasoning}

[[ ## blurb ## ]]
{blurb}        # note: the value you produce must adhere to the JSON schema: {"type": "object", "properties": {"about_book": {"type": "string", "description": "What book contains", "title": "About Book"}, "about_topic": {"type": "string", "description": "Why the topic of book is worth learning", "title": "About Topic"}, "learning_objectives": {"type": "array", "description": "4-6 learning objectives that complete the sentence: You will learn how to ___", "items": {"type": "string"}, "title": "Learning Objectives"}, "target_audience": {"type": "array", "description": "Roles such as Data Engineer, Dat